In [38]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
import string
import MeCab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import unicodedata
from gensim import corpora
from gensim import models
from pprint import pprint

In [121]:
def symbol_removal(soup):
    soup = unicodedata.normalize("NFKC", soup)
    exclusion = "「」『』【】《》、。・◇◆" + "\n" + "\r" + "\u3000"
    soup = soup.translate(str.maketrans("", "", string.punctuation  + exclusion))
    return soup

In [3]:
url = "https://news.yahoo.co.jp/topics/top-picks"
URL = "https://news.yahoo.co.jp/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

In [4]:
all_page_links = []
all_page_links.append(url)
while True:
    try:
        next = soup.find("li", class_="pagination_item-next").find("a")["href"]
        next_link = URL + next
        all_page_links.append(next_link)
        next_res = requests.get(next_link)
        soup = BeautifulSoup(next_res.text, "html.parser")
    except:
        break

In [5]:
all_page_links[5]

'https://news.yahoo.co.jp//topics/top-picks?page=6'

In [6]:
# title_list = []
# category_list = []
# text_list = []
# for i, url in enumerate(all_page_links): # all_page_links: 全てのニュースのリスト
#     if i == 5:
#         res = requests.get(url) # url: 25個分のニュースのリスト
#         soup = BeautifulSoup(res.text, "html.parser")
#         page_soup = soup.find_all("a", class_="newsFeed_item_link")
#         for href in page_soup:
#             link = href["href"] # link: 一つのニュースのリンク(本文は一部のみ)
# #             print(link)
#             link_res = requests.get(link)
#             href_soup = BeautifulSoup(link_res.text, "html.parser")
#             title = href_soup.find("h1", class_=re.compile("^sc")).string
#             title_link = href_soup.find("a", class_="sc-eAyhxF")["href"] # title_link: 本文
# #             print(title_link)
#             res = requests.get(title_link)
#             soup = BeautifulSoup(res.text, "html.parser")
# #             print(soup.find("p", class_=re.compile("^sc")))
            
#             category = soup.find_all("li", class_="current")
# #             print(category)
# #             category = category[1].string
#             if category == []:
# #                 print(category)
#                 continue
#             else:
#                 for tag in soup.find_all(["a"]):
#                     tag.decompose()
# #                 print(soup.find_all("p", class_="yjDirectSLinkTarget")[0].get_text())
#                 try:
#                     text_list.append(soup.find_all("p", class_="yjDirectSLinkTarget")[0].get_text())
#                     title_list.append(title)
# #                     print(category)
#                     print(category[1].string)
# #                     category_list.append(category)
# #                     print("<", title, ">")
#                 except:
#                     continue

In [7]:
title_list = []
category_list = []
text_list = []
for i, url in enumerate(all_page_links): # all_page_links: 全てのニュースのリスト
#     if i == 16:
        res = requests.get(url) # url: 25個分のニュースのリスト
        soup = BeautifulSoup(res.text, "html.parser")
        page_soup = soup.find_all("a", class_="newsFeed_item_link")
        for href in page_soup:
            link = href["href"] # link: 一つのニュースのリンク(本文は一部のみ)
#             print(link)
            link_res = requests.get(link)
            href_soup = BeautifulSoup(link_res.text, "html.parser")
            title = href_soup.find("h1", class_=re.compile("^sc")).string
            title_link = href_soup.find("a", class_="sc-eAyhxF")["href"] # title_link: 本文
#             print(title_link)
            res = requests.get(title_link)
            soup = BeautifulSoup(res.text, "html.parser")
#             print(soup.find("p", class_=re.compile("^sc")))
            
            category = soup.find_all("li", class_="current")
#             print(category)
            try:
                category = category[1].string
            except:
                continue
#             if category == []:
#                 print(category)
                continue
            else:
                for tag in soup.find_all(["a"]):
                    tag.decompose()
#                 print(soup.find_all("p", class_="yjDirectSLinkTarget")[0].get_text())
                try:
                    print(i, title)
                    soup = soup.find_all("p", class_="yjDirectSLinkTarget")[0].get_text()
                    soup = symbol_removal(soup)
#                     print(soup)
                    text_list.append(soup)
#                     text_list.append(soup.find_all("p", class_="yjDirectSLinkTarget")[0].get_text())
                    title_list.append(title)
#                     print(category)
#                     print(category)
                    category_list.append(category)
#                     print("<", title, ">")
                except:
                    continue

0 内村航平が新型コロナ陽性
0 くそたれ地震 不満のみ込む今
0 GoToイート紙切れ 担当者憤り
0 週末 東日本で冷え込み強まる
0 上原氏 田沢の結果あまりに酷
0 SB周東が12戦連続盗塁 プロ新
0 TDR運営 511億円赤字の見通し
0 伊藤容疑者 事故直後の映像
0 日本初 糖質0ビールどう実現
0 給食で100人超食中毒 鶏に菌
0 ビシエド抹消 打点王は絶望的
0 案里被告 参院本会議を欠席
0 「THE W」決勝進出10組が決定
0 巨人 きょう優勝の条件は
0 ニトリ 島忠にTOB実施を発表
0 鈴木氏の出馬断念 知事は驚き
0 東京都 新たに221人感染確認
0 伊藤容疑者 とんかつDJは公開
0 トヨタ、9月生産が過去最高
0 GoToイベント 第1弾にUSJ
0 歓楽街に検査拠点を 分科会
0 AIでいじめ対策 7市導入検討
0 国内感染者 客船含め10万人超
0 デンソー中間決算赤字711億円
0 伊藤容疑者 目撃者が追い説得
1 日銀 大規模緩和の維持決定
1 伊藤容疑者逮捕 事務所が謝罪
1 フジ 伊藤容疑者番組差し替え
1 行き過ぎ指導 元生徒が提訴
1 再審無罪 宮田浩喜さんが死去
1 李元大統領に懲役17年確定
1 第一生命 企業年金利率下げる
1 伊藤容疑者 事故現場離れた
1 エビ中・安本彩花 治療で休養
1 男性不妊 語る治療の難しさ
1 スノボ五輪銅 平岡被告に有罪
1 伊藤健太郎容疑者を逮捕
1 政府 多機能イージス艦新造へ
1 家に犬164匹 多頭飼育崩壊か
1 伊藤健太郎さん事故 女性重傷
1 競技場の盗撮 対策の難しさ
1 首里城火災1年あの日の悔しさ
1 米 WTO事務局長の選出を阻止
1 行き過ぎ指導 元生徒が提訴へ
1 ニトリ、島忠の全株取得へ
1 仏 ロックダウン再導入へ
1 鬼滅 ファンが生んだ「聖地」
1 鈴木大地氏 知事選出馬を断念
1 深田恭子 新婚生活を「妄想」
1 GoTo 電子クーポン不正取得か
2 YouTuber事務所VAZ 社長退任
2 NY株943ドル安 3カ月ぶり安値
2 阪神一筋の上本博紀 退団へ
2 白石 卒コンで何度も「幸せ」
2 錦織圭 右肩負傷で今季終了
2 白石 卒コン序盤から涙腺崩壊
2 かんぽ不正 新たに490人処分
2 関テレ デヴィ夫人発言で謝罪
2 4連敗

In [8]:
print(len(title_list))
print(len(category_list))
print(len(text_list))

496
496
496


In [9]:
df = pd.DataFrame({'title': title_list, 'category': category_list, 'text': text_list})

In [10]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [15]:
ls /work/data/BunshuOnline/

news.csv


In [16]:
df.to_csv('/work/data/BunshuOnline/yahoo_news.csv')

In [234]:
df = pd.read_csv('/work/data/BunshuOnline/yahoo_news.csv', index_col=0)

In [235]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [236]:
df['category'].unique()

array(['スポーツ', '地域', '国内', '経済', 'ライフ', 'エンタメ', '国際', '科学', 'IT'],
      dtype=object)

In [237]:
def sortedTFIDF(sentences):

    # 単語にIDを添付します。
    dictionary = corpora.Dictionary(sentences)

    # 作品ごとの単語の出現回数をカウント
    corpus = list(map(dictionary.doc2bow, sentences))

    # 単語ごとにTF-IDFを算出
    test_model = models.TfidfModel(corpus)
    corpus_tfidf = test_model[corpus]

    # ID:TF-IDF → TF-IDF:単語 に変換。TF-IDFを左に持ってくることで、sortedを用いてTF-IDFを基準にソートすることができます。
    texts_tfidf = []
    for doc in corpus_tfidf:
        text_tfidf = []
        for word in doc:
            text_tfidf.append([word[1], dictionary[word[0]]])
        texts_tfidf.append(text_tfidf)

    # TF-IDFを基準にソートを行います。
    sorted_texts_tfidf = []
    for text in texts_tfidf:
        sorted_text = sorted(text, reverse=True)
        sorted_texts_tfidf.append(sorted_text)

    return sorted_texts_tfidf

In [238]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [239]:
mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/')
wakati = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/")

In [240]:
def MecabMorphologicalAnalysis(path='./text.txt', output_file='wakati.txt', mecab=mecab, file=False):
    mecab_text = ''
    if file:
        with open(path) as f:
            for line in f:
                mecab_text += mecab.parse(line)
        with open(output_file, 'w') as f:
            print(mecab_text, file=f)
    else:
        for path in path.split('\n'):
            mecab_text += mecab.parse(path)
        return mecab_text

In [241]:
texts = []
for i in range(len(df)):
    texts.append(MecabMorphologicalAnalysis(df['text'][i], mecab=wakati))

In [242]:
for i, text in enumerate(texts):
    if i < 5:
        display(text[:100])

'国際体操連盟 FIG の 渡辺 守成 会長 ら が 29日 オンライン で 緊急会見 を 開き 体操 男子 の 種目 別 鉄棒 で 来夏 の 東京五輪 を 目指し て いる 内村航平 リンガーハット '

'死者 44人 建物 被害 が 5 万 棟 に 上っ た 北海道胆振東部地震 から 2年 が たった 被害 の 大半 を 占める 厚真 むか わ 安平 の 被災 3 町 の 仮設住宅 は 多い とき で'

'新型コロナウイルス の 影響 で 打撃 を 受け た 飲食 業界 の 支援策 Go To イート 事業 の プレミアム 付き 食事 券 を 巡り 京都 GoTo イート 事務局 は 28日 京都府内 '

'明日 30日 金 は 上空 の 寒気 が 日本列島 の 上空 に 南下 し て き ます 明日 夜 から は 特に 関東 や 東海 など 東日本 を 中心 に 冷え込み が 強まる 見込み です 31'

'ソフトバンクロッテ 29日 ペイペイドーム ソフトバンク 周東 が 12試合 連続 盗塁 の プロ野球 新記録 を 樹立 し た 初回 に 今季 48 個 目 の 盗塁 と なる 二 盗 に 成功 1'

In [243]:
# リストに変換
sentences = []
for text in texts:
    text_list = text.split(' ')
    sentences.append(text_list)

In [244]:
# まずdocumentsを作り、モデルに渡す。
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1)

In [245]:
for i, doc in enumerate(documents):
#     if i < 10:
        print(doc[1], df['title'][i], df['category'][i], doc[0][:4])

[0] 内村航平が新型コロナ陽性 スポーツ ['国際体操連盟', 'FIG', 'の', '渡辺']
[1] くそたれ地震 不満のみ込む今 地域 ['死者', '44人', '建物', '被害']
[2] GoToイート紙切れ 担当者憤り 地域 ['新型コロナウイルス', 'の', '影響', 'で']
[3] 週末 東日本で冷え込み強まる 国内 ['明日', '30日', '金', 'は']
[4] SB周東が12戦連続盗塁 プロ新 スポーツ ['ソフトバンクロッテ', '29日', 'ペイペイドーム', 'ソフトバンク']
[5] TDR運営 511億円赤字の見通し 経済 ['オリエンタルランド', 'は', '29日', '2021年']
[6] 伊藤容疑者 事故直後の映像 国内 ['29日', 'ひき逃げ', 'など', 'の']
[7] 日本初 糖質0ビールどう実現 ライフ ['糖質ゼロ', 'と', 'いえ', 'ば']
[8] 給食で100人超食中毒 鶏に菌 国内 ['東京都', 'は', '29日', '中野']
[9] ビシエド抹消 打点王は絶望的 スポーツ ['中日', 'の', 'ダヤンビシエド', '内野手']
[10] 案里被告 参院本会議を欠席 国内 ['昨年7月', 'の', '参院選', '広島選挙区']
[11] 「THE W」決勝進出10組が決定 エンタメ ['女性芸人', 'の', '中', 'から']
[12] 巨人 きょう優勝の条件は スポーツ ['巨人', 'は', 'リーグ2', '連覇']
[13] ニトリ 島忠にTOB実施を発表 経済 ['家具', '製造', '販売', '大手']
[14] 鈴木氏の出馬断念 知事は驚き 地域 ['来年', '春', 'の', '千葉県知事選']
[15] 東京都 新たに221人感染確認 国内 ['東京都', 'に', 'よる', 'と']
[16] 伊藤容疑者 とんかつDJは公開 エンタメ ['道交法違反', 'ひき逃げ', 'など', 'の']
[17] トヨタ、9月生産が過去最高 経済 ['トヨタ自動車', 'が', '29日', '発表']
[18] GoToイベント 第1弾にUSJ 経済 ['経済産業省', 'は', '29日', '新型コロナウイルス']
[19] 歓楽

In [246]:
ranking = model.docvecs.most_similar(12, topn=50)

In [247]:
ranking[:5]

[(3, 0.9989606142044067),
 (470, 0.9989197254180908),
 (183, 0.9988960027694702),
 (11, 0.9988939762115479),
 (473, 0.9988885521888733)]

In [248]:
for i, sentence in enumerate(sentences):
    if i < 5:
        print(sentence[:10])

['国際体操連盟', 'FIG', 'の', '渡辺', '守成', '会長', 'ら', 'が', '29日', 'オンライン']
['死者', '44人', '建物', '被害', 'が', '5', '万', '棟', 'に', '上っ']
['新型コロナウイルス', 'の', '影響', 'で', '打撃', 'を', '受け', 'た', '飲食', '業界']
['明日', '30日', '金', 'は', '上空', 'の', '寒気', 'が', '日本列島', 'の']
['ソフトバンクロッテ', '29日', 'ペイペイドーム', 'ソフトバンク', '周東', 'が', '12試合', '連続', '盗塁', 'の']


In [249]:
sorted_texts_tfidf = sortedTFIDF(sentences)

In [250]:
for i, tfidf in enumerate(sorted_texts_tfidf):
    if i < 5:
        print('%s.' % i, '〜%s〜' % df['title'][i]) # 一応タイトルも表示
        pprint(tfidf[:10])
        print('')

0. 〜内村航平が新型コロナ陽性〜
[[0.4004730910293216, '内村'],
 [0.2668113479413732, '渡辺'],
 [0.2668113479413732, '判定'],
 [0.22246929211130856, '陽性'],
 [0.22246929211130856, 'PCR検査'],
 [0.2002365455146608, '国際大会'],
 [0.15551191840717013, '日本代表'],
 [0.15077000608081126, '会長'],
 [0.1331496048534248, '男子'],
 [0.12595054785379373, '21日']]

1. 〜くそたれ地震 不満のみ込む今〜
[[0.17446718208379255, '被害'],
 [0.1712247778368126, '迫り'],
 [0.1712247778368126, '見え隠れ'],
 [0.1712247778368126, '経済的'],
 [0.1712247778368126, '細る'],
 [0.1712247778368126, '立ち退き'],
 [0.1712247778368126, '空き家'],
 [0.1712247778368126, '独り'],
 [0.1712247778368126, '災害救助法'],
 [0.1712247778368126, '後ろめた']]

2. 〜GoToイート紙切れ 担当者憤り〜
[[0.31563662902276013, '券'],
 [0.28390962738246955, '冊'],
 [0.27280495382081965, '食事'],
 [0.22712770190597567, '発券'],
 [0.21252350204590223, '不足'],
 [0.21042441934850675, 'イート'],
 [0.19176161403045197, 'ロール'],
 [0.17551967590850925, '事務局'],
 [0.17034577642948176, '入金'],
 [0.14146435150209483, '手続き']]

3. 〜週末 東日本で冷え込み強まる〜
[[0.517706

In [251]:
all_title = []
for tfidf in sorted_texts_tfidf:
    title = []
    for word in tfidf[:100]: # 100語に絞る
        title.append(word[1])
    all_title.append(title)

In [252]:
# all_titleリストの中身を確認
for i, text in enumerate(all_title):
    if i == 12:
        print(text[:10])


['引き分け', '巨人', '中日', '予告先発', '負け', '大野', 'リーグ2', 'マジック1', '連覇', '平良']


In [253]:
# Doc2Vecモデルに重要単語リストall_titleを渡します。
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(all_title)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1)

In [254]:
# 結果と照らし合わせやすいように、番号と作品の一覧を表示
for i, doc in enumerate(documents):
    print(doc[1], df['title'][i], df['category'][i], doc[0][:8])

[0] 内村航平が新型コロナ陽性 スポーツ ['内村', '渡辺', '判定', '陽性', 'PCR検査', '国際大会', '日本代表', '会長']
[1] くそたれ地震 不満のみ込む今 地域 ['被害', '迫り', '見え隠れ', '経済的', '細る', '立ち退き', '空き家', '独り']
[2] GoToイート紙切れ 担当者憤り 地域 ['券', '冊', '食事', '発券', '不足', 'イート', 'ロール', '事務局']
[3] 週末 東日本で冷え込み強まる 国内 ['°', 'C', '山沿い', '冷え込み', '10', '関東', '明日', '上空']
[4] SB周東が12戦連続盗塁 プロ新 スポーツ ['盗塁', '周東', 'プロ野球', '記録', '二塁', '成功', '福本豊', '月間']
[5] TDR運営 511億円赤字の見通し 経済 ['赤字', '億', '通期', '東京ディズニーランド', '東京ディズニーシー', '来園', 'オリエンタルランド', '622']
[6] 伊藤容疑者 事故直後の映像 国内 ['伊藤', '容疑者', '容疑', '事故', '伊藤健太郎', '映画', 'ショック', '直後']
[7] 日本初 糖質0ビールどう実現 ライフ ['ビール', '糖質ゼロ', '糖質', '一番搾り', 'いえ', '酒好き', 'ハイボール', '気']
[8] 給食で100人超食中毒 鶏に菌 国内 ['給食', '児童', '鶏肉', '中野', '症状', '発熱', '105人', '保健所']
[9] ビシエド抹消 打点王は絶望的 スポーツ ['中日', '82打点', '復帰', '痛', '抹消', '甲子園', '捕手', '阪神']
[10] 案里被告 参院本会議を欠席 国内 ['被告', '里', '保釈', '案', '本会議', '公判', '参院', '罪']
[11] 「THE W」決勝進出10組が決定 エンタメ ['ブロック', '女王', 'ぼる塾', 'Aマッソ', '組', 'ネタ', 'ゆりやんレトリィバァ', 'ゆりやん']
[12] 巨人 きょう優勝の条件は スポーツ ['引き分け', '巨人', '中日', '予告先発', '負け', '

In [255]:
ranking = model.docvecs.most_similar(12, topn=50)

In [256]:
ranking[:5]

[(362, 0.4354647099971771),
 (388, 0.2890816926956177),
 (137, 0.2863055467605591),
 (304, 0.2834857106208801),
 (102, 0.2657347619533539)]

In [257]:
ranking[-5:]

[(145, 0.10992097854614258),
 (166, 0.10593615472316742),
 (274, 0.10480310022830963),
 (269, 0.10471132397651672),
 (392, 0.09755761921405792)]

In [258]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [259]:
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels

    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])

In [260]:
companies = list(df['title'])
docs = sentences

In [261]:
companies[:5]

['内村航平が新型コロナ陽性',
 'くそたれ地震 不満のみ込む今',
 'GoToイート紙切れ 担当者憤り',
 '週末 東日本で冷え込み強まる',
 'SB周東が12戦連続盗塁 プロ新']

In [269]:
print(docs[12:13])

[['巨人', 'は', 'リーグ2', '連覇', 'へ', '優勝', 'マジック', 'を', '2', 'と', 'し', 'て', 'いる', '29日', 'に', '巨人', '勝利', '2位', 'の', '中日', 'が', '負け', 'か', '引き分け', 'の', '場合', '原監督', '率いる', '巨人', 'の', '2年連続', 'リーグ優勝', 'が', '決まる', '巨人', 'が', '引き分け', 'た', '場合', 'は', '中日', 'が', '勝っ', 'て', 'も', '負け', 'て', 'も', '引き分け', 'て', 'も', 'マジック1', 'に', 'なる', '巨人', 'は', '横浜スタジアム', 'で', 'の', 'DeNA', '戦', 'で', '予告先発', 'は', 'サンチェス', 'と', '平良', '中日', 'は', '甲子園', 'で', 'の', '阪神戦', 'で', '予告先発', 'は', '大野', '雄', 'と', '西', '勇', '両', '球場', 'とも', '午後6時', 'の', '同', '時刻', 'に', '開始', 'と', 'なる', '\n']]


In [263]:
texts = LabeledListSentence(docs, companies)

In [276]:
model = models.Doc2Vec(alpha=0.025, min_count=1,
                       size=100, iter=20, workers=4)

In [277]:
model.build_vocab(texts)

<ipython-input-259-2b4a461fa93f>:8: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


In [278]:
model.train(texts, total_words=3409, epochs=10)

<ipython-input-259-2b4a461fa93f>:8: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


In [279]:
len(model.wv.vocab)

17088

In [282]:
model.most_similar(positive=['引き分け'])

<ipython-input-282-e2b1de5dc248>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['引き分け'])


[('女性', 0.9961899518966675),
 ('られる', 0.9961897134780884),
 ('れる', 0.9961504936218262),
 ('\n', 0.9961191415786743),
 ('被告', 0.9961146712303162),
 ('も', 0.9961131811141968),
 ('おり', 0.9961051940917969),
 ('発表', 0.9960916042327881),
 ('不妊', 0.9960893988609314),
 ('昨年', 0.996088981628418)]